In [610]:
#Importando as bibliotecas

from datetime import date
from datetime import timedelta
from datetime import timedelta, time
import pandas as pd
import numpy as np

In [612]:
#Ajustando as colunas conforme o documento - o tasy não cria as colunas de forma que seja possível ler em python, é necessário criar manualmente. 
#copiar as colunas do documento txt e criar a lista a seguir, sem essa etapa o arquivo csv não vai criar as colunas.

columns = ['Nr atendimento',	'Setor paciente',	'Setor prescricao',	'Ds local estoque',	'Cd material',	'Ds material',	'Nr sequencia',	'Dt prescricao',	'Dt entrega setor',	'Dt recebimento setor',	'Dt geracao lote',	'Ds classificacao',	'Dt ger lote',	'Hr ger lote',	'Dt entr setor',	'Hr entr setor',	'Usuario entrega']

colunas_para_retirar = ['Dt recebimento setor', 'Dt ger lote', 'Dt entr setor']

#Ajustando os tipos de dados das colunas conforme o documento
#Facilita deixar cada coluna em uma linha, para ajustar a visualização dos tipos de dados - datas precisam estar como str (será convertido depois). 

tipos_dados = {
    'Nr atendimento': np.int64, 
    'Setor paciente': str, 
    'Setor prescricao': str,  
    'Ds local estoque': str, 
    'Cd material': int, 
    'Ds material': str, 
    'Nr sequencia': float, 
    'Dt prescricao': str, 
    'Dt entrega setor': str, 
    'Dt recebimento setor': str, 
    'Dt geracao lote': str, 
    'Ds classificacao': str, 
    'Dt ger lote': str, 
    'Hr ger lote': str, 
    'Dt entr setor': str, 
    'Hr entr setor': str, 
    'Usuario entrega': str}

In [614]:
#Criando o dataframe#
#Pontos de atenção# 
#Quando ler csv - delimitador precisda estar de acordo, a maioria das vezes o tasy vai importar como tab - usar o argumento delimiter='\t'
#Header vai precisar ficar como None - talvez seja necessário retirar manualmente os cabeçalhos do arquivo original (usar bloco de notas)
#Os argumentos names e dtypes vão puxar da célula acima
#Por algum motivo é necessário definir o engine manualmente - nesse caso deixar como 'python' - **ainda não testei em outros arquivos csv importados do tasy
#Por algum motivo é necessário definir o encoding manualmente - nesse caso deixar como 'latin1' - **ainda não testei em outros arquivos csv importados do tasy

tempos_bruto = pd.read_csv('julho1.csv', delimiter='\t', header=None, names=columns, dtype=tipos_dados, engine='python', encoding='latin1') 


In [616]:
# ajustando o formato das datas # 
# Pontos de atenção #
# Usar o formato - format='%d/%m/%Y\t%H:%M:%S' para datas no formato "01/07/2024 10:16:00"
# Ainda não descobri como arrumar as colunas que tem 0024 - 'Dt ger lote','Dt entr setor' - o python assume que é do século 21. **Essas colunas não são essenciais, acho que não vou procurar essa informação. 
 
tempos_bruto['Dt prescricao'] = pd.to_datetime(tempos_bruto['Dt prescricao'], format='%d/%m/%Y\t%H:%M:%S', errors='coerce')
tempos_bruto['Dt recebimento setor'] = pd.to_datetime(tempos_bruto['Dt recebimento setor'], format='%d/%m/%Y\t%H:%M:%S', errors='coerce')
tempos_bruto['Dt geracao lote'] = pd.to_datetime(tempos_bruto['Dt geracao lote'], format='%d/%m/%Y\t%H:%M:%S', errors='coerce')
tempos_bruto['Dt entrega setor'] = pd.to_datetime(tempos_bruto['Dt entrega setor'], format='%d/%m/%Y\t%H:%M:%S', errors='coerce')

#As horas saem no formato com data (1900-01-01) é necessário criar uma nova coluna sem essa informação com a função .dt.time

tempos_bruto['Hr ger lote'] = pd.to_datetime(tempos_bruto['Hr ger lote'], format='%H:%M:%S', errors='coerce')
tempos_bruto['Hora de geração'] = tempos_bruto['Hr ger lote'].dt.time
tempos_bruto['Hr entr setor'] = pd.to_datetime(tempos_bruto['Hr entr setor'], format='%H:%M:%S', errors='coerce') 
tempos_bruto['Hora de Entrega'] = tempos_bruto['Hr entr setor'].dt.time

In [618]:
# Limpar valores vazios - Normalmente saem valores vazios em colunas "sem lote" ou seja Nr sequencia = 0 
# ** Provavelmente são itens jogados na conta, preciso verificar o que acontece aqui
# Retirei todas as colunas desnecessárias - 'Dt recebimento setor', 'Dt entr setor', 'Dt ger lote','Hr ger lote','Hr entr setor' 
## Como não descobri como arrumar a questão do 0024 e as colunas não são necessárias, retirei - 'Dt ger lote','Dt entr setor'


Tempos_Limpo0 = tempos_bruto.dropna(subset=['Ds classificacao'])
Tempos_Limpo = Tempos_Limpo0.drop(columns=['Dt recebimento setor', 'Dt entr setor', 'Dt ger lote','Hr ger lote','Hr entr setor']) 


In [620]:
                                                                    ## MEDIANA ##

# Fazendo os cálculos base para MEDIANA #
# Antes de iniciar o calculo é necessário usar o timedelta nas horas. 
from datetime import timedelta # Import the timedelta object
Tempos_Limpo['Hora de geração'] = pd.to_timedelta(Tempos_Limpo['Hora de geração'].astype(str))
Tempos_Limpo['Hora de Entrega'] = pd.to_timedelta(Tempos_Limpo['Hora de Entrega'].astype(str))

# Criar a coluna nova com o tempo de entrega. 
Tempos_Limpo['Tempo de entrega'] = (Tempos_Limpo['Hora de Entrega'] - Tempos_Limpo['Hora de geração']).apply(lambda x: str(x)[-8:])

# Converter a coluna 'Tempo de entrega' para timedelta64[ns]
Tempos_Limpo['Tempo de entrega'] = pd.to_timedelta(Tempos_Limpo['Tempo de entrega'])

## Filtrando para apenas os itens na Urgência - 'agora' ##

#Selecionar o valor do filtro
valor_desejado = 'Agora'  

# Filtrar o dataframe para 'agora'
Tempos_Limpo_Urgencia = Tempos_Limpo[Tempos_Limpo['Ds classificacao'].isin([valor_desejado])].copy()  # Create a copy of the filtered DataFrame

# Converter 'Tempo de entrega' para minutos usando .loc para garantir segurança
Tempos_Limpo_Urgencia.loc[:, 'Tempo de entrega (minutos)'] = Tempos_Limpo_Urgencia['Tempo de entrega'].dt.total_seconds() / 60

#  Remover NaN do 'Tempo de entrega (minutos)'
Tempos_Limpo_Urgencia = Tempos_Limpo_Urgencia.dropna(subset=['Tempo de entrega (minutos)'])

# Calcular a mediana em segundos
mediana_tempo_entrega_segundos = Tempos_Limpo_Urgencia['Tempo de entrega'].dt.total_seconds().median()

# Converter mediana para objeto timedelta
mediana_tempo_entrega_urgencias = timedelta(seconds=mediana_tempo_entrega_segundos)

# Extrair horas, minutos, and segundoss do objeto timedelta
hours = mediana_tempo_entrega_urgencias.seconds // 3600
minutes = (mediana_tempo_entrega_urgencias.seconds % 3600) // 60
seconds = mediana_tempo_entrega_urgencias.seconds % 60

# Mostrar a mediana em horas, minutos, and segundos
f"Mediana do Tempo de entrega de urgências - Diurno e Noturno: {hours:02}:{minutes:02}:{seconds:02}"



'Mediana do Tempo de entrega de urgências - Diurno e Noturno: 00:25:06'

In [622]:
                                                            ##  MEDIANA - PERÍODO NOTURNO ##

# Filtrar por 'Hora de geração' antes das 06:00:00 ou depois 22:00:00
hora_limite_inferior = time(6, 0, 0)
hora_limite_superior = time(22, 0, 0)

# Colocar 'Hora de geração' no formato timedelta
Tempos_Limpo_Urgencia['Hora de geração'] = pd.to_timedelta(Tempos_Limpo_Urgencia['Hora de geração'].astype(str))

# Extrair a hora da geração, Substituir NaT por 0
hora_geracao_noturno = Tempos_Limpo_Urgencia['Hora de geração'].dt.components.hours.fillna(0) 

# Aplicar o filtro usando as horas limite (note que noturno tem | e diurno tem &, o que garante a filtragem correta)
filtered_indices = (hora_geracao_noturno < hora_limite_inferior.hour) | (hora_geracao_noturno > hora_limite_superior.hour)

# Filtrar o dataframe usando uma mascara boleana
Tempos_Limpo_Urgencia_noturno = Tempos_Limpo_Urgencia[filtered_indices]

# Aplicar o filtro para Tempos_Limpo_Urgencia
Tempos_Limpo_Urgencia_noturno = Tempos_Limpo_Urgencia[filtered_indices]

# Calcular o tempo de entrega em segundos
mediana_tempo_entrega_segundos_noturno = Tempos_Limpo_Urgencia_noturno['Tempo de entrega'].dt.total_seconds().median()

# Converter a mediana para objeto timedelta
mediana_tempo_entrega_noturno = timedelta(seconds=mediana_tempo_entrega_segundos_noturno)

# Extrair horas, minutos, and segundoss do objeto timedelta
hours_noturno = mediana_tempo_entrega_noturno.seconds // 3600
minutes_noturno = (mediana_tempo_entrega_noturno.seconds % 3600) // 60
seconds_noturno = mediana_tempo_entrega_noturno.seconds % 60

# Mostrar a mediana em horas, minutos, and segundos
f"Mediana do Tempo de entrega (Noturno): {hours_noturno:02}:{minutes_noturno:02}:{seconds_noturno:02}"

'Mediana do Tempo de entrega (Noturno): 00:24:55'

In [624]:
                                                        ## MEDIANA - PERÍODO DIURNO ##

# Filtrar por 'Hora de geração' antes das 06:00:00 ou depois 22:00:00
hora_limite_inferior = time(6, 0, 0)
hora_limite_superior = time(22, 0, 0)

# Colocar 'Hora de geração' no formato timedelta
Tempos_Limpo_Urgencia['Hora de geração'] = pd.to_timedelta(Tempos_Limpo_Urgencia['Hora de geração'].astype(str))

# Extrair a hora da geração, Substituir NaT por 0
hora_geracao_noturno = Tempos_Limpo_Urgencia['Hora de geração'].dt.components.hours.fillna(0)  

# Aplicar o filtro usando as horas limite (note que noturno tem | e diurno tem &, o que garante a filtragem correta)
filtered_indices_business_hours = (hora_geracao_noturno >= hora_limite_inferior.hour) & (hora_geracao_noturno <= hora_limite_superior.hour)

# Filtrar o dataframe usando uma mascara boleana
Tempos_Limpo_Urgencia_Diurno = Tempos_Limpo_Urgencia[filtered_indices_business_hours]

# Calcular o tempo de entrega em segundos
mediana_tempo_entrega_segundos_diurno = Tempos_Limpo_Urgencia_Diurno['Tempo de entrega'].dt.total_seconds().median()

# Converter a mediana para objeto timedelta
mediana_tempo_entrega_segundos_diurno = timedelta(seconds=mediana_tempo_entrega_segundos_diurno)

# Extrair horas, minutos, and segundoss do objeto timedelta
hours_diurno = mediana_tempo_entrega_segundos_diurno.seconds // 3600
minutes_diurno = (mediana_tempo_entrega_segundos_diurno.seconds % 3600) // 60
seconds_business_diurno = mediana_tempo_entrega_segundos_diurno.seconds % 60

# Mostrar a mediana em horas, minutos, and segundos
f"Mediana do Tempo de entrega (Diurno): {hours_diurno:02}:{minutes_diurno:02}:{seconds_business_diurno:02}"

'Mediana do Tempo de entrega (Diurno): 00:25:08'

In [630]:
mediana_tempo_entrega_Urgencias = [f"Mediana do Tempo de entrega de urgências - Diurno e Noturno: {hours:02}:{minutes:02}:{seconds:02}",
    f"Mediana do Tempo de entrega (Noturno): {hours_noturno:02}:{minutes_noturno:02}:{seconds_noturno:02}",
    f"Mediana do Tempo de entrega (Diurno): {hours_diurno:02}:{minutes_diurno:02}:{seconds_business_diurno:02}"]

In [640]:
# TESTES #

# Dataframes criados #

#1 - tempos_bruto
#2 - Tempos_Limpo0 - Retirado valores nulos do nr sequencia (número do lote)
#3 - Tempos_Limpo - Retirado todas as colunas desnecessárias - 'Dt recebimento setor', 'Dt entr setor', 'Dt ger lote','Hr ger lote','Hr entr setor' 
#4 - Tempos_Limpo_Urgencia - Filtrado apenas pro 'Agora'
#5 - Tempos_Limpo_Urgencia_Diurno - Filtrado para 'Agora' e Horário 06 até 22h
#6 - Tempos_Limpo_Urgencia_noturno - Filtrado para 'Agora' e Horário antes 06 ou após 22h

# Calculos criados #

# 1 - mediana_tempo_entrega_urgencias 
# 2 - mediana_tempo_entrega_noturno 
# 3 - mediana_tempo_entrega_segundos_diurno 
# 4 - mediana_tempo_entrega_Urgencias - lista com todos os tempos

mediana_tempo_entrega_Urgencias

['Mediana do Tempo de entrega de urgências - Diurno e Noturno: 00:25:06',
 'Mediana do Tempo de entrega (Noturno): 00:24:55',
 'Mediana do Tempo de entrega (Diurno): 00:25:08']